# 03 - Neural Network Baseline
This notebook builds a simple neural network regression model using PyTorch.
We aim to compare its performance against the Lasso regression model.

## Step 1: Load Cleaned Data

In [3]:
import pandas as pd

# Load processed dataset
df = pd.read_csv("../../data/processed/cleaned_framingham.csv")
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


##  Step 2: Define Features and Target

In [4]:
# Define target and features
X = df.drop('TenYearCHD', axis=1)
y = df['TenYearCHD']

## Step 3: Train-Test Split & Scaling

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Step 4: Define Neural Network in PyTorch

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define model
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)


##  Step 5: Train the Model

In [7]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Model setup
model = SimpleNN(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 10/100, Loss: 0.1450
Epoch 20/100, Loss: 0.1327
Epoch 30/100, Loss: 0.1262
Epoch 40/100, Loss: 0.1217
Epoch 50/100, Loss: 0.1188
Epoch 60/100, Loss: 0.1168
Epoch 70/100, Loss: 0.1153
Epoch 80/100, Loss: 0.1142
Epoch 90/100, Loss: 0.1132
Epoch 100/100, Loss: 0.1125


## Step 6: Evaluate Model Performance

In [8]:
# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).squeeze()
    mse = criterion(predictions, y_test_tensor.squeeze())
    print(f"Test MSE: {mse.item():.4f}")

Test MSE: 0.1250


## Step 7: Save Model

In [9]:
torch.save(model.state_dict(), "../models/simple_nn_model.pt")

✅ Neural network training complete. Model and performance saved.

Next step: Develop a hybrid model combining Lasso feature selection with NN correction.